In [ ]:
#!pip install tweet-preprocessor

In [2]:
import preprocessor as p
import pandas as pd
import json
import re

In [3]:
pd.options.display.max_colwidth = 75

### Turn query documents (tweets) into generators

In [4]:
def load_tweets(file):
    with open(file, 'r') as f:
        tweets = (json.loads(line) for line in f.readlines())
    return tweets

In [7]:
cd

/home/jovyan


In [13]:
cd ../capstone-35/gulf_twitter_raw/

/home/jovyan/capstone-35/gulf_twitter_raw


In [14]:
query_gen = load_tweets('stream_الطريج.jsonl')

### Turn query generators into dataframes

In [15]:
def tweet_to_df(tweets):
    data = {'text': [], 'screen_name': [], 'geo': [],
       }
    
    for t in tweets:
    
        data['text'].append(t['text'])
        data['screen_name'].append(t['user']['screen_name'])
        data['geo'].append(t['user']['location'])
    
    return pd.DataFrame(data)

In [16]:
query_df = tweet_to_df(query_gen)

In [17]:
query_df.info() 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 454 entries, 0 to 453
Data columns (total 3 columns):
geo            301 non-null object
screen_name    454 non-null object
text           454 non-null object
dtypes: object(3)
memory usage: 10.7+ KB


### Clean tweets (remove emojis, links, http, etc.)

In [ ]:
def cleaner(message):
    message = re.sub("([0-9A-Za-z:!/.])", " ", message)
    return message

In [ ]:
cleaned_text = []

for tweet in query_df['text']:
    cleaned_text.append(p.clean(cleaner(tweet)))

query_df['cleaned_text']=cleaned_text

In [ ]:
cleaned_name = []

for tweet in query_df['screen_name']:
    cleaned_name.append(p.clean(tweet))

query_df['cleaned_name']=cleaned_name

In [ ]:
cleaned_geo = []

for tweet in query_df.geo:
    cleaned_geo.append(p.clean(str(tweet)))

query_df['cleaned_geo']=cleaned_geo


In [ ]:
query_df.sample(4)

### Remove duplicated tweets

In [ ]:
query_df['cleaned_text'].duplicated().sample(5)

In [ ]:
query_cleaned = query_df.drop_duplicates(['cleaned_text'], keep='last')
query_cleaned = query_cleaned.drop(['screen_name', 'text', 'geo'], axis=1)

In [ ]:
query_cleaned.shape

In [ ]:
query_cleaned.sample(5)

In [ ]:
query_cleaned.cleaned_geo.value_counts().head()

In [ ]:
query_cleaned.cleaned_name.value_counts().head()

In [ ]:
query_cleaned.fillna('None')

In [ ]:
query_cleaned.info() 

### Consolidate different geo tags

In [18]:
locations = {'Saudi Arabia': ['Kingdom of Saudi Arabia', 'MAKKAH', 'saudi arabia', 'jeddah', 'مكة المكرمة', 'المملكة العربية', 'Jeddah', 'Riyadh', 'المملكة العربية السعودية', 'الرياض, المملكة العربية السعودية', 'الرياض, المملكة العربية السعودية', 'jeddah , saudi arabia', 'الرياض', 'آلقصيم ، بريده', 'جدة, المملكة العربية السعودية', 'المدينة المنورة, المملكة العربية السعودية', 'Riyadh, Kingdom of Saudi Arabia', 'makkah - saudi arabia', 'جده', 'Dammam', 'Dammam, Eastern', 'Al Khobar, Kingdom of Saudi Arabia'],
             'Kuwait': ['الكويت', 'Alkuwait', 'kuwait', 'KUWAIT', 'Kuwait , Salwa', 'Qortuba, Kuwait', 'دولة الكويت', 'العاصمه, دولة الكويت', 'The Capital, Kuwait', 'Al Salam, Kuwait', 'Salwa, Kuwait', 'كوكب الشرق - Kuwait'],
             'Egypt': ['Domiat, Egypt', 'Damitta', 'Alexandria, Egypt', 'Damitta-Egypt', 'Giza, Egypt', 'New damieta city', 'مصر', 'القاهرة, مصر', 'El Behera, Egypt', 'الجيزة, مصر', 'Elmansoura', 'El Menia, Egypt', 'El Qaliobia, Egypt', 'البحيرة, مصر', 'Domyat elgadeda', 'El Sharkia, Egypt', 'sharkia_fakous', 'Cairo, Egypt', 'Alexandria,Egypt', 'ELsharkia , Egypt', 'شمال سيناء, مصر', 'كفر الشيخ, مصر', 'Port Said, Egypt', 'Beni Suef, Egypt', 'القليوبية, مصر', 'بنها']
            }

In [ ]:
'MAKKAH' in list(locations.values())[0]

In [ ]:
'MAKKAH' in list(locations.values())[1]

In [ ]:
locations.values()

In [ ]:
#pseudo code

for word in query_cleaned['cleaned_geo'].str.split('_'):
    if word[0] in list(locations.values())[0]:
        query_cleaned['cleaned_geo'].replace(word, 'Saudi Arabia', inplace=True)
    else:
        if word[0] in list(locations.values())[1]:
            query_cleaned['cleaned_geo'].replace(word, 'Kuwait', inplace=True)

In [ ]:
query_cleaned.cleaned_geo

In [ ]:
query_cleaned.info()

In [ ]:
query_cleaned.sample(5)

### Pickle to conduct EDA in seperate notebook

In [ ]:
pwd

In [ ]:
query_cleaned.to_pickle('../gulf_twitter_pickled/stream_الطريج.p')